- 트랜스포머(Transformer)는 2017년 구글이 발표한 논문인 "Attention is all you need"에서 나온 모델로 기존의 seq2seq의 구조인 인코더-디코더를 따르면서도, 논문의 이름처럼 어텐션(Attention)만으로 구현한 모델입니다. 이 모델은 RNN을 사용하지 않고, 인코더-디코더 구조를 설계하였음에도 성능도 RNN보다 우수하다는 특징을 갖고있습니다.

- BERT가 높은 성능을 얻을 수 있었던 것은, 레이블이 없는 방대한 데이터로 사전 훈련된 모델을 가지고, 레이블이 있는 다른 작업(Task)에서 추가 훈련과 함께 하이퍼파라미터를 재조정하여 이 모델을 사용하면 성능이 높게 나오는 기존의 사례들을 참고하였기 때문입니다. 다른 작업에 대해서 파라미터 재조정을 위한 추가 훈련 과정을 파인 튜닝(Fine-tuning)이라고 합니다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
!pip install transformers

BERT는 문맥을 반영한 임베딩(Contextual Embedding)을 사용

![dd](https://wikidocs.net/images/page/115055/bert0.PNG)

# 4. BERT의 서브워드 토크나이저 : WordPiece

In [3]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

'Here is the sentence I want embeddings for.'라는 문장을 BERT의 토크나이저가 어떻게 토큰화하는지 봅시다.



In [4]:
result = tokenizer.tokenize('Here is the sentence I want embeddings for.')
print(result)

['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']


embeddings라는 단어는 단어 집합에 존재하지 않으므로 em, ##bed, ##ding, #s로 분리됩니다. 실제로 BERT의 단어 집합에 특정 단어가 있는지 조회하려면 .vocab[]을 통해서 가능합니다. 우선 단어 'here'을 조회해봅시다.

여기서 ##은 이 서브워드들은 단어의 중간부터 등장하는 서브워드라는 것을 알려주기 위해 단어 집합 생성 시 표시해둔 기호입니다. 이런 표시가 있어야만 em, ##bed, ##ding, #s를 다시 손쉽게 embeddings로 복원할 수 있을 것입니다.

In [5]:
print(tokenizer.vocab['here'])

2182


In [7]:
# embeddings 단어 조회해보자
print(tokenizer.vocab['embeddings'])

KeyError: ignored

In [9]:
# 그렇다면 em, ##bed, ##ding, ##s 는 있을까?
print(tokenizer.vocab['em'])
print(tokenizer.vocab['##bed'])
print(tokenizer.vocab['##ding'])
print(tokenizer.vocab['##s'])

7861
8270
4667
2015


In [10]:
# BERT의 단어집합 전체를 불러와 저장해보자
## vocabulary.txt에 저장

with open('vocabulary.txt', 'w') as f:
  for token in tokenizer.vocab.keys():
    f.write(token + '\n')

In [11]:
df = pd.read_fwf('vocabulary.txt', header = None)
df

,0
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]
...,...
30517,##．
30518,##／
30519,##：
30520,##？


In [12]:
# tokenizer.vocab이 가지고 있는 dictionary 크기 확인
len(df)

30522

In [13]:
# '##ding' 조회해보자
df.loc[4667].values[0]

'##ding'

참고로 BERT에서 사용되는 특별 토큰들과 그와 맵핑되는 정수는 다음과 같습니다.

  * [PAD] - 0
  * [UNK] - 100
  * [CLS] - 101
  * [SEP] - 102
  * [MASK] - 103

In [14]:
# 임의로 102번 토큰만 출력
df.loc[102].values[0]

'[SEP]'

# 5. 포지션 임베딩(Position Embedding)

트랜스포머에서는 포지셔널 인코딩(Positional Encoding)이라는 방법을 통해서 단어의 위치 정보를 표현했습니다. 포지셔널 인코딩은 사인 함수와 코사인 함수를 사용하여 위치에 따라 다른 값을 가지는 행렬을 만들어 이를 단어 벡터들과 더하는 방법입니다. BERT에서는 이와 유사하지만, 위치 정보를 사인 함수와 코사인 함수로 만드는 것이 아닌 **학습을 통해서 얻는 포지션 임베딩(Position Embedding)이라는 방법을 사용**합니다.


![dd](https://wikidocs.net/images/page/115055/%EA%B7%B8%EB%A6%BC5.PNG)

위의 그림은 포지션 임베딩을 사용하는 방법을 보여줍니다. 우선, 위의 그림에서 WordPiece Embedding은 우리가 이미 알고 있는 단어 임베딩으로 실질적인 입력입니다. 그리고 이 입력에 포지션 임베딩을 통해서 **위치 정보**를 더해주어야 합니다. 포지션 임베딩의 아이디어는 굉장히 간단한데, 위치 정보를 위한 **임베딩 층(Embedding layer)을 하나 더 사용**합니다. 가령, 문장의 길이가 4라면 4개의 포지션 임베딩 벡터를 학습시킵니다. 그리고 BERT의 입력마다 다음과 같이 포지션 임베딩 벡터를 더해주는 것입니다.

* 첫번째 단어의 임베딩 벡터 + 0번 포지션 임베딩 벡터
* 두번째 단어의 임베딩 벡터 + 1번 포지션 임베딩 벡터
* 세번째 단어의 임베딩 벡터 + 2번 포지션 임베딩 벡터
* 네번째 단어의 임베딩 벡터 + 3번 포지션 임베딩 벡터

실제 BERT에서는 문장의 최대 길이를 512로 하고 있으므로, 총 512개의 포지션 임베딩 벡터가 학습됩니다. **결론적으로 현재 설명한 내용을 기준으로는 BERT에서는 총 두 개의 임베딩 층이 사용됩니다**. 단어 집합의 크기가 30,522개인 단어 벡터를 위한 임베딩 층과 문장의 최대 길이가 512이므로 512개의 포지션 벡터를 위한 임베딩 층입니다.

사실 BERT는 세그먼트 임베딩(Segment Embedding)이라는 1개의 임베딩 층을 더 사용합니다. 이에 대해서는 뒤에 언급합니다.

# 6. 사전 훈련(Pre-training)

위의 그림은 BERT의 논문에 첨부된 그림으로 ELMo와 GPT-1, 그리고 BERT의 구조적인 차이를 보여줍니다. 가장 우측 그림의 ELMo는 정방향 LSTM과 역방향 LSTM을 각각 훈련시키는 방식으로 양방향 언어 모델을 만들었습니다. 가운데 그림의 GPT-1은 트랜스포머의 디코더를 이전 단어들로부터 다음 단어를 예측하는 방식으로 단방향 언어 모델을 만들었습니다. Trm은 트랜스포머를 의미합니다. 단방향(→)으로 설계된 Open AI GPT와 달리 가장 좌측 그림의 BERT는 화살표가 양방향으로 뻗어나가는 모습을 보여줍니다. 이는 마스크드 언어 모델(Masked Language Model)을 통해 양방향성을 얻었기 때문입니다. BERT의 사전 훈련 방법은 크게 두 가지로 나뉩니다. 첫번째는 마스크드 언어 모델이고, 두번째는 다음 문장 예측(Next sentence prediction, NSP)입니다.

논문에 따르면 BERT는 BookCorpus(8억 단어)와 위키피디아(25억 단어)로 학습되었습니다.

## 마스크드 언어 모델(Masked Language Model, MLM)

BERT는 ***사전 훈련을 위해서 인공 신경망의 입력으로 들어가는 입력 텍스트의 15%의 단어를 랜덤으로 마스킹(Masking)합니다.*** 그리고 인공 신경망에게 이 가려진 단어들을(Masked words) 예측하도록 합니다. 중간에 단어들에 구멍을 뚫어놓고, 구멍에 들어갈 단어들을 예측하게 하는 식입니다. 예를 들어 '나는 [MASK]에 가서 그곳에서 빵과 [MASK]를 샀다'를 주고 '슈퍼'와 '우유'를 맞추게 합니다.

더 정확히는 전부 [MASK]로 변경하지는 않고, 랜덤으로 선택된 15%의 단어들은 다시 다음과 같은 비율로 규칙이 적용됩니다.

* 80%의 단어들은 [MASK]로 변경한다.
Ex) The man went to the store → The man went to the [MASK]

* 10%의 단어들은 랜덤으로 단어가 변경된다.
Ex) The man went to the store → The man went to the dog

* 10%의 단어들은 동일하게 둔다.
Ex) The man went to the store → The man went to the store

이렇게 하는 이유는 [MASK]만 사용할 경우에는 [MASK] 토큰이 파인 튜닝 단계에서는 나타나지 않으므로 사전 학습 단계와 파인 튜닝 단계에서의 불일치가 발생하는 문제가 있습니다. 이 문제을 완화하기 위해서 랜덤으로 선택된 15%의 단어들의 모든 토큰을 [MASK]로 사용하지 않습니다. 이를 전체 단어 관점에서 그래프를 통해 정리하면 다음과 같습니다.

![dd](https://wikidocs.net/images/page/115055/%EC%A0%84%EC%B2%B4%EB%8B%A8%EC%96%B4.PNG)


'My dog is cute. he likes playing'이라는 문장에 대해서 마스크드 언어 모델을 학습하고자 합니다. 약간의 전처리와 BERT의 서브워드 토크나이저에 의해 이 문장은 ['my', 'dog', 'is' 'cute', 'he', 'likes', 'play', '##ing']로 토큰화가 되어 BERT의 입력으로 사용됩니다. 그리고 언어 모델 학습을 위해서 다음과 같이 데이터가 변경되었다고 가정해봅시다.

* 'dog' 토큰은 [MASK]로 변경되었습니다.

![sa](https://wikidocs.net/images/page/115055/%EA%B7%B8%EB%A6%BC8.PNG)

위 그림은 'dog' 토큰이 [MASK]로 변경되어서 BERT 모델이 원래 단어를 맞추려고 하는 모습을 보여줍니다. 여기서 출력층에 있는 다른 위치의 벡터들은 예측과 학습에 사용되지 않고, 오직 'dog' 위치의 출력층의 벡터만이 사용됩니다. 구체적으로는 BERT의 손실 함수에서 다른 위치에서의 예측은 무시합니다. 출력층에서는 예측을 위해 단어 집합의 크기만큼의 밀집층(Dense layer)에 소프트맥스 함수가 사용된 1개의 층을 사용하여 원래 단어가 무엇인지를 맞추게 됩니다. 그런데 만약 'dog'만 변경된 것이 아니라 다음과 같이 데이터셋이 변경되었다면 어떨까요? 이번에는 세 가지 유형 모두에 대해서 가정해봅시다.

* 'dog' 토큰은 [MASK]로 변경되었습니다.
* 'he'는 랜덤 단어 'king'으로 변경되었습니다.
* 'play'는 변경되진 않았지만 예측에 사용됩니다.


## 2) 다음 문장 예측(Next Sentence Prediction, NSP)

BERT는 두 개의 문장을 준 후에 이 문장이 이어지는 문장인지 아닌지를 맞추는 방식으로 훈련시킵니다. 이를 위해서 50:50 비율로 실제 이어지는 두 개의 문장과 랜덤으로 이어붙인 두 개의 문장을 주고 훈련시킵니다. 이를 각각 Sentence A와 Sentence B라고 하였을 때, 다음의 예는 문장의 연속성을 확인한 경우와 그렇지 않은 경우를 보여줍니다.

* 이어지는 문장의 경우
Sentence A : The man went to the store.
Sentence B : He bought a gallon of milk.
Label = IsNextSentence

* 이어지는 문장이 아닌 경우 경우
Sentence A : The man went to the store.
Sentence B : dogs are so cute.
Label = NotNextSentence

![ss](https://wikidocs.net/images/page/115055/%EA%B7%B8%EB%A6%BC10.PNG)

BERT의 입력으로 넣을 때에는 [SEP]라는 특별 토큰을 사용해서 문장을 구분합니다. 첫번째 문장의 끝에 [SEP] 토큰을 넣고, 두번째 문장이 끝나면 역시 [SEP] 토큰을 붙여줍니다. 그리고 이 두 문장이 실제 이어지는 문장인지 아닌지를 [CLS] 토큰의 위치의 출력층에서 이진 분류 문제를 풀도록 합니다. [CLS] 토큰은 BERT가 분류 문제를 풀기 위해 추가된 특별 토큰입니다. 그리고 위의 그림에서 나타난 것과 같이 마스크드 언어 모델과 다음 문장 예측은 따로 학습하는 것이 아닌 loss를 합하여 학습이 동시에 이루어집니다.

BERT가 언어 모델 외에도 다음 문장 예측이라는 태스크를 학습하는 이유는 BERT가 풀고자 하는 태스크 중에서는 QA(Question Answering)나 NLI(Natural Language Inference)와 같이 두 문장의 관계를 이해하는 것이 중요한 태스크들이 있기 때문입니다.

# 구글 BERT의 마스크드 언어 모델(Masked Language Model)실습

## 마스크드 언어 모델
transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. *BERT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다.* 예를 들어서 A라는 이름의 BERT를 사용하는데, B라는 이름의 BERT의 토크나이저를 사용하면 모델은 텍스트를 제대로 이해할 수 없습니다. A라는 BERT의 토크나이저는 '사과'라는 단어를 36번으로 정수 인코딩하는 반면에, B라는 BERT의 토크나이저는 '사과'라는 단어를 42번으로 정수 인코딩하는 등 단어와 맵핑되는 정수 정보 자체가 다르기 때문입니다.

In [43]:
from transformers import TFBertForMaskedLM, AutoTokenizer, BertTokenizer

In [41]:
# 모델 로드
model = TFBertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

## BERT의 입력
''Soccer is a really fun [MASK]'라는 임의의 문장이 있다고 해봅시다. 이를 마스크드 언어 모델의 입력으로 넣으면, 마스크드 언어 모델은 [MASK]의 위치에 해당하는 단어를 예측합니다. 마스크드 언어 모델의 예측 결과를 보기위해서 bert-large-uncased의 토크나이저를 사용하여 해당 문장을 정수 인코딩해봅시다.

In [42]:
text = 'Soccer is a really fun.'

In [17]:
input = tokenizer(text, return_tensors = 'tf')
# 모델의 입력
print(input['input_ids'])

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


In [18]:
# 토크나이저로 변환된 결과에서 token_type_ids를 통해 문장을 구분짓는 id
print(input['token_type_ids'])

## 현재의 입력은 문장이 두 개가 아니라 한 개이므로 여기서는 문장 길이만큼의 0 시퀀스를 얻습니다. 
## 만약 문장이 두 개였다면 두번째 문장이 시작되는 구간부터는 1의 시퀀스가 나오게 되지만, 여기서는 해당되지 않습니다.

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


In [20]:
print(input['attention_mask'])

##현재의 입력에서는 패딩이 없으므로 여기서는 문장 길이만큼의 1 시퀀스를 얻습니다. 
## 만약 뒤에 패딩이 있었다면 패딩이 시작되는 구간부터는 0의 시퀀스가 나오게 되지만, 여기서는 해당되지 않습니다.

tf.Tensor([[1 1 1 1 1 1 1 1 1]], shape=(1, 9), dtype=int32)


## Tokenize 함수
  - 입력된 문장을 바로 tokenizing

In [44]:
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

input_ids = tokenizer.encode(text)
print(input_ids)

decoded_ids = tokenizer.decode(input_ids)
print(decoded_ids)

['Soccer', 'is', 'a', 'really', 'fun', '.']
[101, 25914, 10124, 169, 30181, 41807, 119, 102]
[CLS] Soccer is a really fun. [SEP]


## Truncation
  - 음절이나 어절 단위로 자르는 것이 아니라 입력된 문장을 tokenizing했을 때의 token들을 기준으로 자른다
  - 원래의 입력 문장과 길이와 관련이 없다

In [45]:
# 5개의 token만 살리고 뒤는 제거
tokenized_text = tokenizer.tokenize(text, add_special_tokens = False, max_length = 5, truncation = True)
print(tokenized_text)

input_ids = tokenizer.encode(text, add_special_tokens = False, max_length = 5, truncation = True)
print(input_ids)

decode_ids = tokenizer.decode(input_ids)
print(decoded_ids)

['Soccer', 'is', 'a', 'really', 'fun']
[25914, 10124, 169, 30181, 41807]
[CLS] Soccer is a really fun. [SEP]


## Padding
  - option이 다양하다
    - 문장의 앞 또는 뒤 중 어디에 padding을 부착할 것인가
    - 문장 두개가 입력으로 주어진 경우, 첫 번재 문장을 padding해서 maximum length를 채울 것인지 두 번째 문장을 padding해서 채울것인지 결정
    - pad가 부착된것은 0의 vocab을 가진다

In [46]:
print(tokenizer.pad_token)
print(tokenizer.pad_token_id)

[PAD]
0


In [47]:
tokenized_text = tokenizer.tokenize(
    text,
    add_special_tokens=False,
    max_length=20,
    padding="max_length"
    )
print(tokenized_text)

input_ids = tokenizer.encode(
    text,
    add_special_tokens=False,
    max_length=20,
    padding="max_length"
    )
print(input_ids)

decoded_ids = tokenizer.decode(input_ids)
print(decoded_ids)

['Soccer', 'is', 'a', 'really', 'fun', '.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[25914, 10124, 169, 30181, 41807, 119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Soccer is a really fun. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [49]:
text = 'Soccer is a really fun [MASK].'
tokenized_text = tokenizer.tokenize(text)

print(tokenized_text)

from transformers import pipeline
# 'bert-base-multilingual-cased' 모델로 학습
nlp_fill = pipeline('fill-mask', model = 'bert-base-multilingual-cased')
nlp_fill('Soccer is a really fun [MASK].')

['Soccer', 'is', 'a', 'really', 'fun', '[MASK]', '.']


Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.38734906911849976,
  'sequence': 'Soccer is a really fun thing.',
  'token': 40414,
  'token_str': 'thing'},
 {'score': 0.11343719065189362,
  'sequence': 'Soccer is a really fun sport.',
  'token': 17925,
  'token_str': 'sport'},
 {'score': 0.0863523930311203,
  'sequence': 'Soccer is a really fun game.',
  'token': 11661,
  'token_str': 'game'},
 {'score': 0.02384401112794876,
  'sequence': 'Soccer is a really fun fun.',
  'token': 41807,
  'token_str': 'fun'},
 {'score': 0.010257362388074398,
  'sequence': 'Soccer is a really fun deal.',
  'token': 19918,
  'token_str': 'deal'}]

In [50]:
# 'bert-large-uncased' 모델로 학습
nlp_fill2 = pipeline('fill-mask', model = 'bert-large-uncased')
nlp_fill2('Soccer is a really fun [MASK].')

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.7621229290962219,
  'sequence': 'soccer is a really fun sport.',
  'token': 4368,
  'token_str': 'sport'},
 {'score': 0.2034219205379486,
  'sequence': 'soccer is a really fun game.',
  'token': 2208,
  'token_str': 'game'},
 {'score': 0.012208702974021435,
  'sequence': 'soccer is a really fun thing.',
  'token': 2518,
  'token_str': 'thing'},
 {'score': 0.0018630443373695016,
  'sequence': 'soccer is a really fun activity.',
  'token': 4023,
  'token_str': 'activity'},
 {'score': 0.001335499226115644,
  'sequence': 'soccer is a really fun field.',
  'token': 2492,
  'token_str': 'field'}]

# 3. [MASK] 토큰 예측하기

FillMaskPipeline은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리해서 보여줍니다. FillMaskPipeline에 우선 앞서 불러온 모델과 토크나이저를 지정해줍니다.

In [21]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model = model, tokenizer = tokenizer)

In [26]:
# 입력 문장으로부터 [MASK]의 위치에 들어갈 수 있는 상위 5개의 후보 단어들을 출력해보자

pip('Soccer is a really fun [MASK].')

[{'score': 0.9753891229629517,
  'sequence': 'soccer is a really fun.',
  'token': 1012,
  'token_str': '.'},
 {'score': 0.014161502942442894,
  'sequence': 'soccer is a really fun!',
  'token': 999,
  'token_str': '!'},
 {'score': 0.00993503537029028,
  'sequence': 'soccer is a really fun ;',
  'token': 1025,
  'token_str': ';'},
 {'score': 0.00047694580280222,
  'sequence': 'soccer is a really fun?',
  'token': 1029,
  'token_str': '?'},
 {'score': 1.8595414076116867e-05,
  'sequence': 'soccer is a really fun...',
  'token': 2133,
  'token_str': '...'}]

# 한국어 BERT의 마스크드 언어 모델(Masked Language Model) 실습

## 1. BERT의 입력

'축구는 정말 재미있는 [MASK]다' 라는 임의의 문장이 있다고 해봅시다. 를 마스크드 언어 모델의 입력으로 넣으면, 마스크드 언어 모델은 [MASK]의 위치에 해당하는 단어를 예측합니다. 마스크드 언어 모델의 예측 결과를 보기위해서 klue/bert-base의 토크나이저를 사용하여 해당 문장을 정수 인코딩해봅시다.


In [51]:
# 해당 모델이 기존에는 tf가 아니라 pytorch로 학습된 모델이었지만 이를 tf에서 사용
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt = True)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [52]:
input = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors = 'tf')

In [53]:
# 토크나이저로 변환된 결과에서 input_ids를 통해 정수인코딩 결과를 확인
print(input['input_ids'])

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


In [54]:
# 세그먼트 인코딩 결과 확인
print(input['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


In [55]:
# 실제 단어와 패딩 토큰을 구분하는 용도인 어텐션 마스크를 확인
print(input['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


## 3. [MASK] 토큰 예측하기

In [56]:
pip = FillMaskPipeline(model = model, tokenizer = tokenizer)

In [57]:
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.896351158618927,
  'sequence': '축구는 정말 재미있는 스포츠 다.',
  'token': 4559,
  'token_str': '스포츠'},
 {'score': 0.025957664474844933,
  'sequence': '축구는 정말 재미있는 거 다.',
  'token': 568,
  'token_str': '거'},
 {'score': 0.010033938102424145,
  'sequence': '축구는 정말 재미있는 경기 다.',
  'token': 3682,
  'token_str': '경기'},
 {'score': 0.00792439840734005,
  'sequence': '축구는 정말 재미있는 축구 다.',
  'token': 4713,
  'token_str': '축구'},
 {'score': 0.007844222709536552,
  'sequence': '축구는 정말 재미있는 놀이 다.',
  'token': 5845,
  'token_str': '놀이'}]

In [58]:
pip('어벤져스는 정말 재미있는 [MASK]다.')

[{'score': 0.83824223279953,
  'sequence': '어벤져스는 정말 재미있는 영화 다.',
  'token': 3771,
  'token_str': '영화'},
 {'score': 0.02827564999461174,
  'sequence': '어벤져스는 정말 재미있는 거 다.',
  'token': 568,
  'token_str': '거'},
 {'score': 0.017189430072903633,
  'sequence': '어벤져스는 정말 재미있는 드라마 다.',
  'token': 4665,
  'token_str': '드라마'},
 {'score': 0.014989713206887245,
  'sequence': '어벤져스는 정말 재미있는 이야기 다.',
  'token': 3758,
  'token_str': '이야기'},
 {'score': 0.009382630698382854,
  'sequence': '어벤져스는 정말 재미있는 장소 다.',
  'token': 4938,
  'token_str': '장소'}]

In [59]:
pip('나는 오늘 아침에 [MASK]에 출근을 했다.')

[{'score': 0.08012565970420837,
  'sequence': '나는 오늘 아침에 회사 에 출근을 했다.',
  'token': 3769,
  'token_str': '회사'},
 {'score': 0.061240967363119125,
  'sequence': '나는 오늘 아침에 에 출근을 했다.',
  'token': 1,
  'token_str': '[UNK]'},
 {'score': 0.017486680299043655,
  'sequence': '나는 오늘 아침에 공장 에 출근을 했다.',
  'token': 4345,
  'token_str': '공장'},
 {'score': 0.016131814569234848,
  'sequence': '나는 오늘 아침에 사무실 에 출근을 했다.',
  'token': 5841,
  'token_str': '사무실'},
 {'score': 0.015360785648226738,
  'sequence': '나는 오늘 아침에 서울 에 출근을 했다.',
  'token': 3671,
  'token_str': '서울'}]

In [ ]:
!pip install transformers

In [46]:
import os
import torch
import transformers
import random
import numpy as np
import time
import datetime

from transformers import BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [16]:
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
  print(torch.cuda.get_device_name(i))

1
Tesla K80


In [2]:
!git clone https://github.com/e9t/nsmc.git 

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 14.15 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [5]:
import pandas as pd


train = pd.read_csv('nsmc/ratings_train.txt', sep = '\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


## 전처리
  - BERT 분류모델은 각 문장의 앞마다 [CLS]를 붙여 인식한다
  - 문장 종료는 [SEP]으로 알린다

In [6]:
document_bert = ['[CLS] ' + str(s) + " [SEP]" for s in train.document]
document_bert[:5]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]']

## Toeknizing
  - 사전 학습된 BERT multilingual 모델 내 포함되어있는 토크나이저를 활용해 토크나이징

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case = False)
tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]
print(tokenized_texts[0])

['[CLS]', '아', '더', '##빙', '.', '.', '진', '##짜', '짜', '##증', '##나', '##네', '##요', '목', '##소', '##리', '[SEP]']


## 패딩
  - token들의 max_length보다 크게 max_len을 설정한다
  - 설정한 max_len 만큼 빈 공간을 0으로 채운다

In [22]:
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen = MAX_LEN, dtype = 'long', truncating = 'post', padding = 'post')
input_ids[0]

array([   101,   9519,   9074, 119005,    119,    119,   9708, 119235,
         9715, 119230,  16439,  77884,  48549,   9284,  22333,  12692,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

## 어텐션 마스크
  - 학습속도를 높이기 위해 실 데이터가 있는 곳과 padding이 있는 곳을 attention에게 알려준다

In [23]:
attention_masks = []

for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## train-validation set 분리
  - input과 mask가 뒤섞이지 않도록 random_state를 일정하게 고정
  - test set은 위에서 이미 분리되었기에 train과 validation set만 분리

In [28]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, 
                                                                                    train['label'].values,
                                                                                    random_state = 42,
                                                                                    test_size = 0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks,
                                                       input_ids,
                                                       random_state = 42,
                                                       test_size = .1)

## 파이토치 텐서로 변환
  - numpy ndarray로 되어있는 input, label, mask들을 torch tensor로 변환

In [29]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

## 배치 및 데이터로더 설정
  - 현재 쓰고 있는 GPU의 VRAM에 맞도록 배치사이즈 설정
  - 우선 배치사이즈를 크게 넣어보고 VRAM 부족 메시지가 나오면 8의 배수중 더 작은 것으로 줄여나가는 것이 일반적인 방법이다

In [31]:
BATCH_SIZE = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader= DataLoader(validation_data, sampler = validation_sampler, batch_size = BATCH_SIZE)

## 테스트셋 전처리
  - 위의 train-val셋 전처리와 동일
  - train 전처리시 함께 처리해도 무방

In [32]:
sentences = test['document']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = test['label'].values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

# 모델 학습
## GPU 체크 및 할당

In [33]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 분류를 위한 BERT모델 생성
  - transformers의 BertForSequenceClassification 모듈 이용
  - 이진분류이므로 num_labels는 2

In [35]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels = 2)
model.cuda()

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

## 학습 스케쥴링
  - transformers에서 제공하는 옵티마이저 중 AdamW를 사용
  - 총 훈련 스텝은 이터레이션 * 에폭수로 설정
  - lr 스케쥴러는 역시 transformers 에서 제공하는 것으로 사용

In [37]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)

epochs= 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# 학습

## accuracy와 시간 표시함수 정의

In [48]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis = 1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
  elapsed_rounded = int(round(elapsed))
  return str(datetime.timedelta(seconds = elapsed_rounded))

## 학습 실행
  - 데이터로더에서 배치만큼 가져온 후 forward, backward pass를 수행
  - gradient update는 명시적으로 하지않고 위에서 로드한 optimizer를 활용

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


## 테스트셋 평가

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))